In [ ]:
from tqdm.notebook import tqdm as progress
import pandas as pd
import plotly.express as px
from plotly.offline import plot
import dimcli
from dimcli.utils import *
import os, sys, time, json
from IPython.display import Image
from IPython.core.display import HTML 
import csv
from itertools import islice
import ast

In [ ]:
#Log into Dimensions
#API key is: 9F8D648F0D7E437CB1736BEBDF007F02
#!pip install dimcli -U --quiet 

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
    import getpass
    KEY = getpass.getpass(prompt='API Key: ')  
    dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
    KEY = "9F8D648F0D7E437CB1736BEBDF007F02"
    dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()

In [ ]:
singleRead = {}
file_path = r'\Users\ashle\OneDrive\School\Capstone\dimensionsAI\nameOneIDs.csv'
with open(file_path, 'r') as csv_file:
    # Create a CSV DictReader object
    csv_reader = csv.DictReader(csv_file)

    # Iterate through the rows in the CSV file
    for row in csv_reader:
        # Each row is now a dictionary with column names as keys
        singleRead[row['Name']] = ast.literal_eval(row['ID'])
        
        
first_half_items = list(singleRead.items())
# Create a new dictionary using the first half items
whole_Dict = dict(first_half_items)

print(len(whole_Dict))
print(whole_Dict)


In [ ]:
search_Var2 = "[abstract + acronym + active_years + altmetric + associated_grant_ids + brief_title + category_bra + category_for + category_for_2020 + category_hra + category_hrcs_hc + category_hrcs_rac + category_icrp_cso + category_icrp_ct + category_rcdc + conditions + date + date_inserted + dimensions_url + end_date + funder_countries + funders + gender + id + interventions + investigators + linkout + mesh_terms + phase + publication_ids + publications + registry + research_orgs + researchers + score + start_date + study_arms + study_designs + study_eligibility_criteria + study_maximum_age + study_minimum_age + study_outcome_measures + study_participants + study_type + title]"

# var2 = "abstract,acronym,active_years,altmetric,associated_grant_ids,brief_title,category_bra,category_for,category_for_2020,category_hra,category_hrcs_hc,category_hrcs_rac,category_icrp_cso,category_icrp_ct,category_rcdc,conditions,date,date_inserted,dimensions_url,end_date,funder_countries,funders,gender,id,interventions,investigators,linkout,mesh_terms,phase,publication_ids,publications,registry,research_orgs,researchers,score,start_date,study_arms,study_designs,study_eligibility_criteria,study_maximum_age,study_minimum_age,study_outcome_measures,study_participants,study_type,title"
# var2 = str(var2.replace(",", " + "))
# search_Var2 = var2
# print(var2)

In [ ]:
# study_arms, study_designs, study_eligibility_criteria, study_maximum_age, study_minimum_age, study_outcome_measures, study_participants, study_type
# abstract, conditions, gender, linkout, phase registry, category_bra, category_for, category_hra, category_hrcs_hc, category_hrcs_rac, category_icrp_cso, category_icrp_ct, category_rcdc + active_years + id + investigators + title


def findby_IdCT(nameDict):
    all_CT= {}
    count=0
    for name, person_id in nameDict.items():
        print(person_id[0])        
        res = dsl.query_iterative(f"""search clinical_trials where researchers = "{person_id[0]}" return clinical_trials {search_Var2}""")
        number = res.json['_stats']['total_count']
        print(f'{person_id[0]} id has {number} clinical trials')
        listDict = res.json['clinical_trials']
        name_list = []
        for ct in listDict:
            name_list.append(ct)
        
        all_CT[name] = name_list
        count+=1
        print(f'{count}. {name}')
  
    return all_CT

ct = findby_IdCT(whole_Dict) 

# Specify the file name
json_filename = "all_ClinicalTrials.json"

# Writing the dictionary to a JSON file
with open(json_filename, 'w') as json_file:
    json.dump(ct, json_file, indent=2)

print(f'The data has been written to {json_filename}')